In [ ]:
import os

# Server Side
class QuicServer:
    # Other methods remain unchanged

    def process_data(self, data):
        """Process incoming data on the stream."""
        # For demonstration purposes, just print the received data
        print("Received data:", data)